# 実行環境の作成

## GCP認証

下記コードでGCPに接続

In [1]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-japan
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-japan statistics-japan

## ライブラリ

In [4]:
import pandas as pd
import plotly.express as px
# Google Colab.でプロットするためには，以下を実行する．
import plotly.io as pio
pio.renderers.default = "colab"

import json
import tqdm
import os

# GBQからデータ取得

## テーブルデータ取得

In [5]:
from google.cloud import bigquery

def get_estat_origin(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'estat_ranking'
  client = bigquery.Client(project=project_id)

  query_string = f'SELECT * FROM {dataset_id}.{table_id}'

  return client.query(query_string).result().to_dataframe()

In [7]:
table_id = '0000010101_A1101'
get_estat_origin(table_id).head()

,index,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,45,A1101,01000,1975100000,5338206,人,総人口,北海道,1975年度,5
1,46,A1101,01000,1976100000,5395000,人,総人口,北海道,1976年度,5
2,47,A1101,01000,1977100000,5443000,人,総人口,北海道,1977年度,5
3,48,A1101,01000,1978100000,5490000,人,総人口,北海道,1978年度,5
4,49,A1101,01000,1979100000,5535000,人,総人口,北海道,1979年度,5


## テーブルデータを結合

In [8]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
    ]
}

In [6]:
import pandas as pd

def df_data(card):
  
  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 結合
  for c in card['categories']:
    df = get_estat_origin(list(c.values())[0])
    df['categoryName'] = list(c.keys())[0]
    df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
    df['rankJapan'] = df.astype(str).apply(lambda x: '全国 第' + x['rankJapan'] + '位', 1)
    df = df.drop('index', axis=1)

    df_res = pd.concat([df_res, df])
  
  return df_res

In [78]:
df_data(card).head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,A1101,01000,1975,5338206,人,総数,北海道,1975年度,全国 第5位
1,A1101,01000,1976,5395000,人,総数,北海道,1976年度,全国 第5位
2,A1101,01000,1977,5443000,人,総数,北海道,1977年度,全国 第5位
3,A1101,01000,1978,5490000,人,総数,北海道,1978年度,全国 第5位
4,A1101,01000,1979,5535000,人,総数,北海道,1979年度,全国 第5位


# 人口・世帯

## ComposedChart

In [7]:
"""
rechartsのComposedChartをJSONに保存する関数（地域ごと）

"""

def save_recharts_composed(cardId,df):

  areas = sorted(list(set(df['areaCode'].tolist())))
  for area in tqdm.notebook.tqdm(areas):
    
    # 地域で抽出
    df_area = df.copy().query(f'areaCode == "{area}"').reset_index()

    # レスポンスを整形
    dic = {
        'catgories': list_categories(df_area),
        'times': list_times(df_area),
        'values': dict_recharts_composed_data(df_area),
    }
    
    # GCSに保存
    gcs_dir = "statistics-japan/recharts/{}".format(cardId)
    os.makedirs(gcs_dir, exist_ok=True)
    with open(f'{gcs_dir}/{area}.json', 'w') as f:
      json.dump(dic,f,ensure_ascii=False)
    
  return

def list_categories(df):
  return df['categoryName'].drop_duplicates().reset_index()['categoryName'].tolist()

def list_times(df):
  return df[['timeCode','timeName']].drop_duplicates(subset=['timeCode']).sort_values('timeCode').to_dict(orient='records')

def dict_recharts_composed_data(df):
  df_value = df.pivot(index='timeCode', columns='categoryName', values='value' ).reset_index()
  df_rank = df.pivot(index='timeCode', columns='categoryName', values='rankJapan' ).reset_index().rename(columns=lambda x: x + '_rankJapan')

  df = pd.merge(df_value, df_rank, left_on='timeCode', right_on='timeCode_rankJapan',how='left')

  return df.to_dict(orient='records')

#### 総人口

In [105]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
        { '15歳未満' : '0000010101_A1301' },
        { '15～64歳' : '0000010101_A1302' },
        { '65歳以上' : '0000010101_A1303' },
    ]
}
df = df_data(card)

In [106]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 結婚・離婚

In [107]:
card = {
    'cardId': 'marriage',
    'categories': [
        { '婚姻件数' : '0000010101_A9101' },
        { '離婚件数' : '0000010101_A9201' },
    ]
}
	
df = df_data(card)
df.head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,A9101,13000,1975,111176,組,婚姻件数,東京都,1975年度,全国 第1位
1,A9101,13000,1976,101939,組,婚姻件数,東京都,1976年度,全国 第1位
2,A9101,13000,1977,94425,組,婚姻件数,東京都,1977年度,全国 第1位
3,A9101,13000,1978,90222,組,婚姻件数,東京都,1978年度,全国 第1位
4,A9101,13000,1979,89599,組,婚姻件数,東京都,1979年度,全国 第1位


In [108]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 年齢調整死亡率

In [16]:
card = {
    'cardId': 'mortality',
    'categories': [
        { '男性' : '0000010101_A424001' },
        { '女性' : '0000010101_A424002' },
    ]
}
df = df_data(card)

In [17]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 年齢中位数

In [11]:
card = {
    'cardId': 'median-age',
    'categories': [
        { '年齢中位数' : '0000010101_A1231' },
    ]
}
df = df_data(card)

In [12]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 昼夜間人口比率

In [8]:
card = {
    'cardId': 'day-night',
    'categories': [
        { '昼夜間人口比率' : '0000010101_A6108' },
    ]
}
df = df_data(card)

In [9]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 社会増減率

In [12]:
card = {
    'cardId': 'social-change-rate',
    'categories': [
        { '社会増減率' : '0000010101_A5301' },
    ]
}
df = df_data(card)

In [13]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 合計特殊出生率

In [15]:
card = {
    'cardId': 'total-fertility-rate',
    'categories': [
        { '合計特殊出生率' : '0000010101_A4103' },
    ]
}
df = df_data(card)

In [16]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 年齢３区分人口

In [17]:
card = {
    'cardId': 'age-population',
    'categories': [
        { '15歳未満' : '0000010101_A1304' },
        { '15～64歳' : '0000010101_A1305' },
        { '65歳以上' : '0000010101_A1306' },
    ]
}
df = df_data(card)

In [18]:
save_recharts_composed(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

## Pyramid

### 年齢別人口

In [98]:
"""
apexchartsのPyramidChartをJSONに保存する関数（地域ごと）

"""
def save_recharts_pyramid(cardId,df):
  
  areas = sorted(list(set(df['areaCode'].tolist())))
  for area in tqdm.notebook.tqdm(areas):

    # 地域のデータを抽出
    df_area = df.copy().query(f'areaCode == "{area}"').reset_index()
    
    # レスポンスを整形
    dic = dict_area(df_area)

    
    # GCSに保存
    gcs_dir = "statistics-japan/recharts/{}".format(cardId)
    os.makedirs(gcs_dir, exist_ok=True)
    with open(f'{gcs_dir}/{area}.json', 'w') as f:
      json.dump(dic,f,ensure_ascii=False)

  return

def dict_area(df):
  return {
        'catgories': list_categories(df),
        'times': list_times(df),
        'values': dict_value(df),
    }

def list_categories(df):
  return df['categoryName'].str.replace('（女）', '').str.replace('（男）', '').drop_duplicates().reset_index()['categoryName'].tolist()

def list_times(df):
  return df[['timeCode','timeName']].drop_duplicates(subset=['timeCode']).sort_values('timeCode').to_dict(orient='records')


def df_man_value(df_arg):
  df = df_arg.query('categoryName.str.contains("男")', engine='python').astype(({"value": object}))[['timeCode','categoryName','value']]
  df = df.rename(columns={'value': '男性'})
  df['categoryName'] = df['categoryName'].str.replace('（男）', '') 
  return df

def df_woman_value(df_arg):
  df = df_arg.query('categoryName.str.contains("女")', engine='python').astype(({"value": object}))[['timeCode','categoryName','value']]
  df = df.rename(columns={'value': '女性'})
  df['categoryName'] = df['categoryName'].str.replace('（女）', '')
  df['女性'] = df['女性'] * -1
  return df

def dict_value(df_arg):
  df_man = df_man_value(df_arg)
  df_woman = df_woman_value(df_arg)

  df = pd.merge(df_man,df_woman, on=['timeCode','categoryName'],how='left').iloc[::-1]
  return df.to_dict(orient='records')


In [99]:
card = {
    'cardId': 'population-pyramid',
    'categories': [
        { "0～4歳（男）": "0000010101_A120101"}, 
        { "0～4歳（女）": "0000010101_A120102"}, 
        { "5～9歳（男）": "0000010101_A120201"}, 
        { "5～9歳（女）": "0000010101_A120202"}, 
        { "10～14歳（男）": "0000010101_A120301"}, 
        { "10～14歳（女）": "0000010101_A120302"}, 
        { "15～19歳（男）": "0000010101_A120401"}, 
        { "15～19歳（女）": "0000010101_A120402"},  
        { "20～24歳（男）": "0000010101_A120501"}, 
        { "20～24歳（女）": "0000010101_A120502"}, 
        { "25～29歳（男）": "0000010101_A120601"}, 
        { "25～29歳（女）": "0000010101_A120602"}, 
        { "30～34歳（男）": "0000010101_A120701"}, 
        { "30～34歳（女）": "0000010101_A120702"}, 
        { "35～39歳（男）": "0000010101_A120801"}, 
        { "35～39歳（女）": "0000010101_A120802"}, 
        { "40～44歳（男）": "0000010101_A120901"}, 
        { "40～44歳（女）": "0000010101_A120902"}, 
        { "45～49歳（男）": "0000010101_A121001"}, 
        { "45～49歳（女）": "0000010101_A121002"}, 
        { "50～54歳（男）": "0000010101_A121101"}, 
        { "50～54歳（女）": "0000010101_A121102"}, 
        { "55～59歳（男）": "0000010101_A121201"}, 
        { "55～59歳（女）": "0000010101_A121202"}, 
        { "60～64歳（男）": "0000010101_A121301"}, 
        { "60～64歳（女）": "0000010101_A121302"}, 
        { "65～69歳（男）": "0000010101_A121401"}, 
        { "65～69歳（女）": "0000010101_A121402"}, 
        { "70～74歳（男）": "0000010101_A121501"}, 
        { "70～74歳（女）": "0000010101_A121502"}, 
        { "75～79歳（男）": "0000010101_A121601"}, 
        { "75～79歳（女）": "0000010101_A121602"}, 
        { "80～84歳（男）": "0000010101_A121701"}, 
        { "80～84歳（女）": "0000010101_A121702"}, 
        { "85～89歳（男）": "0000010101_A121801"}, 
        { "85～89歳（女）": "0000010101_A121802"}, 
        { "90～94歳（男）": "0000010101_A121901"}, 
        { "90～94歳（女）": "0000010101_A121902"}, 
        { "95～99歳（男）": "0000010101_A122001"}, 
        { "95～99歳（女）": "0000010101_A122002"}, 
        { "100歳以上（男）": "0000010101_A122101"}, 
        { "100歳以上（女）": "0000010101_A122102"}
    ]
}
df = df_data(card)

In [100]:
save_recharts_pyramid(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]